# Coordinates

*Author: Creare* <br>
*Date: April 01 2020* <br>

**Keywords**: podpac, Coordinates

## Overview

Reference tutorial for the `podpac.Coordinates` class.

### Prerequisites

- Python 2.7 or above
- [`podpac`](https://podpac.org/install.html#install)
- *Review the [README.md](../README.md) and [jupyter-tutorial.ipynb](jupyter-tutorial.ipynb) for additional info on using jupyter notebooks*

### See Also

- [introduction.ipynb](introduction.ipynb): PODPAC introduction
- [`podpac.Coordinates` API Reference](https://podpac.org/api/podpac.Coordinates.html#podpac.Coordinates)
- [advanced Coordinates notebook](../4-advanced/coordinates.ipynb): More information on Coordinates

## Terminology

**Coordinates** are used to:

1. Evaluate nodes which retrieve and process data
2. Define the coordinates of data sources

PODPAC `Coordinates` are modeled after the `coords` in [xarray](http://xarray.pydata.org/en/stable/data-structures.html), with some additional restrictions and enhancements.

In PODPAC, **coordinates** refer to actual location along an axis for a given **dimension**.

**Dimension** refers to the name of the axis. In PODPAC, the allowed **dimensions** are:

* `'alt'`
* `'lat'`
* `'lon'`
* `'time'`

PODPAC uses the terminology of `stacked` versus `unstacked` coordinates.

* When coordinates are unstacked, each dimension has its own axis
* When coordinates are stacked, every stacked dimension share an axis
    * This means the number of coordinates for each dimension has to be the same
<img src="../../images/unstack-stack.png" style='width:80%;margin-left:auto;margin-right:auto;' />

# Coordinates creation

`Coordinates` are created from a list of coordinate `values` and a corresponding list of `dims`:

```python
podpac.Coordinates(values, dims=dims, ...)
```

Unlike xarray, PODPAC coordinate values are always either `float` or `np.datetime64`.

For convenience, PODPAC automatically converts datetime strings such as `'2018-01-01'` to `np.datetime64`. 

## Unstacked Coordinates

Grid Coordinates

In [1]:
# Create 2D lat, lon grid from a list of coordinates
import podpac

# Create coordinates for dimensions
lat = [0, 1, 2]         # lat dimension
lon = [10, 20, 30, 40]  # lon dimension

# Create PODPAC coordinates
c = podpac.Coordinates([lat, lon], dims=['lat', 'lon'])

# c is a 3x4 grid of points
print ("Grid Shape:", c.shape)
print ("Grid Size:", c.size)
print ('Grid:', c)

Grid Shape: (3, 4)
Grid Size: 12
Grid: Coordinates (EPSG:4326)
	lat: ArrayCoordinates1d(lat): Bounds[0.0, 2.0], N[3]
	lon: ArrayCoordinates1d(lon): Bounds[10.0, 40.0], N[4]


In [2]:
# Create 3D lat, lon, time grid from a list of coordinates
lat = [0, 1, 2]                      # lat dimension
lon = [10, 20, 30, 40]               # lon dimension
time = ['2018-01-01', '2018-01-02']  # time dimension

# Create PODPAC coordinates
c = podpac.Coordinates([lat, lon, time], dims=['lat', 'lon', 'time'])

# c is a 3x4x2 grid of points
print ("Grid Shape:", c.shape)
print ("Grid Size:", c.size)
print ('Grid:', c)

Grid Shape: (3, 4, 2)
Grid Size: 24
Grid: Coordinates (EPSG:4326)
	lat: ArrayCoordinates1d(lat): Bounds[0.0, 2.0], N[3]
	lon: ArrayCoordinates1d(lon): Bounds[10.0, 40.0], N[4]
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-01-02], N[2]


## Stacked Coordinates

`Coordinates` from multiple dimensions can be stacked together in a list (rather than representing a grid).

For example, `Coordinates` with stacked latitude and longitude contain one point for each (lat, lon) pair. 
Note that the name for this stacked dimension is `'lat_lon'`, using an underscore to combine the underlying dimensions.

In [3]:
# Create 1D lat_lon axis from a list of coordinates
lat = [0, 1, 2]     # lat dimension
lon = [10, 20, 30]  # lon dimension

# Create the coordinates, note the nested list
c = podpac.Coordinates([[lat, lon]], dims=[['lat', 'lon']])

# c is a length 3 axis of points
print ("Grid Shape:", c.shape)
print ("Grid Size:", c.size)
print ('Grid:', c)

Grid Shape: (3,)
Grid Size: 3
Grid: Coordinates (EPSG:4326)
	lat_lon[lat]: ArrayCoordinates1d(lat): Bounds[0.0, 2.0], N[3]
	lat_lon[lon]: ArrayCoordinates1d(lon): Bounds[10.0, 30.0], N[3]


## Hybrid unstacked / stacked coordinates

Stacked and unstacked coordinates can be combined in a `Coordinates` object.

In [4]:
# Create coordinates for dimensions
lat = [0, 1, 2]
lon = [10, 20, 30]
time = ['2018-01-01', '2018-01-02']

# Create the coordinates, note the nested list for lon_lat
podpac.Coordinates([[lon, lat], time], dims=[['lon', 'lat'], 'time'])

Coordinates (EPSG:4326)
	lon_lat[lon]: ArrayCoordinates1d(lon): Bounds[10.0, 30.0], N[3]
	lon_lat[lat]: ArrayCoordinates1d(lat): Bounds[0.0, 2.0], N[3]
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-01-02], N[2]

## Uniformly spaced coordinates

Specifying a uniformly-spaced grid allows some optimization in PODPAC.

PODPAC provides two convenience functions `crange` and `clinspace` for creating uniformly-spaced coordinates, similar to the `arange` and `linspace` functions provided by [NumPy](https://www.numpy.org/).

These functions wrap `UniformCoordinates1d` (see Advanced Usage in the [Coordinates developer notebook](../developer/Coordinates.ipynb)), which is particularly useful for coordinates with an extremely large number of points.

## PODPAC crange
`podpac.crange` creates uniformly-spaced coordinates from a *start*, *stop*, and *step*.

In `podpac.crange`:

* string inputs are supported for datetimes and timedeltas
* the stop value will be included in the coordinates if it falls an exact number of steps from the start

In [5]:
# Time coordinates can also be created from strings
podpac.crange('2018-01-01', '2018-03-01', '1,M')  

UniformCoordinates1d(?): Bounds[2018-01-01, 2018-03-01T00:00:00.000000], N[3]

In [6]:
# Note, the (?) above means we didn't give this coordinate a dimension name
c = podpac.crange('2018-01-01', '2018-03-01', '1,M', name='time')
c

UniformCoordinates1d(time): Bounds[2018-01-01, 2018-03-01T00:00:00.000000], N[3]

## Inclusion of the *stop* value

The stop value will be included in the coordinates if it falls an exact number of steps from the start

In [7]:
# Notice 'stop' value IS NOT included!
podpac.crange(0, 7, 2)

UniformCoordinates1d(?): Bounds[0.0, 6.0], N[4]

In [8]:
# Notice 'stop' value IS included!
podpac.crange(0, 8, 2)

UniformCoordinates1d(?): Bounds[0.0, 8.0], N[5]

Regular PODPAC `Coordinates` can be created using `crange`

In [9]:
c = podpac.Coordinates([podpac.crange(90, -90, -1), podpac.crange(-180, 180, 2)], 
                       ['lat', 'lon'])
c

Coordinates (EPSG:4326)
	lat: UniformCoordinates1d(lat): Bounds[-90.0, 90.0], N[181]
	lon: UniformCoordinates1d(lon): Bounds[-180.0, 180.0], N[181]

## PODPAC clinspace

`podpac.clinspace` creates uniformly-spaced coordinates from a *start*, *stop*, and *size*.

In `podpac.clinspace`:

* string inputs are supported for datetimes
* tuple inputs are supported for stacked coordinates

In [10]:
# Time coordinates can be created from strings
podpac.clinspace('2018-01-01', '2018-03-01', 3)

UniformCoordinates1d(?): Bounds[2018-01-01, 2018-03-01T00], N[3]

In [11]:
# Tuple inputs for stacked coordinates: Creates a line between the specified points
podpac.clinspace((0, 10), (1, 20), 3)

StackedCoordinates
	None[?]: UniformCoordinates1d(?): Bounds[0.0, 1.0], N[3]
	None[?]: UniformCoordinates1d(?): Bounds[10.0, 20.0], N[3]

Regular PODPAC `Coordinates` can be created using `clinspace`

In [12]:
c = podpac.Coordinates([podpac.clinspace((0, 10), (1, 20), 3), podpac.clinspace('2018-01-01', '2018-03-01', 3)], 
                       ['lat_lon', 'time'])
c

Coordinates (EPSG:4326)
	lat_lon[lat]: UniformCoordinates1d(lat): Bounds[0.0, 1.0], N[3]
	lat_lon[lon]: UniformCoordinates1d(lon): Bounds[10.0, 20.0], N[3]
	time: UniformCoordinates1d(time): Bounds[2018-01-01, 2018-03-01T00], N[3]

# Coordinates usage

In the course of creating coordinates, its sometimes useful to combine or modify coordinates in a few ways. This section describes how to:
* Drop dimensions from a `Coordinates` instance
* Create new coordinates by merging dimensions from multiple `Coordinates`
* Create new coordinates by concatenating or taking the union of multiple `Coordinates`

## Dropping dimensions
Sometimes it's useful to drop a dimension from a `Coordinates` instance. For example, when trying to create coordinates from the `native_coordinates` of a datasource.

In [13]:
# Create coordinates for the example
c = podpac.Coordinates([[1, 2, 3], [4, 5, 6]], ['lat', 'lon'])
c

Coordinates (EPSG:4326)
	lat: ArrayCoordinates1d(lat): Bounds[1.0, 3.0], N[3]
	lon: ArrayCoordinates1d(lon): Bounds[4.0, 6.0], N[3]

In [14]:
# Drop the latitude dimension
c.drop(['lat'])

Coordinates (EPSG:4326)
	lon: ArrayCoordinates1d(lon): Bounds[4.0, 6.0], N[3]

## Merging dimensions
Coordinates can be created by merging dimension for multiple coordinates.

In [15]:
# Create Coordinates describing space
c_space = podpac.Coordinates([[1, 2, 3], [4, 5, 6, 7]], ['lat', 'lon'])

# Create Coordinates describing time
c_time = podpac.Coordinates([['2018-01-01', '2018-12-12']], ['time'])
print(c_space)
print(c_time)

Coordinates (EPSG:4326)
	lat: ArrayCoordinates1d(lat): Bounds[1.0, 3.0], N[3]
	lon: ArrayCoordinates1d(lon): Bounds[4.0, 7.0], N[4]
Coordinates (EPSG:4326)
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-12-12], N[2]


In [16]:
# Combine the two coordinates
c_combined0 = podpac.coordinates.merge_dims([c_space, c_time])

# Note, order is important
c_combined1 = podpac.coordinates.merge_dims([c_time, c_space])

print (c_combined0)
print (c_combined1)

Coordinates (EPSG:4326)
	lat: ArrayCoordinates1d(lat): Bounds[1.0, 3.0], N[3]
	lon: ArrayCoordinates1d(lon): Bounds[4.0, 7.0], N[4]
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-12-12], N[2]
Coordinates (EPSG:4326)
	time: ArrayCoordinates1d(time): Bounds[2018-01-01, 2018-12-12], N[2]
	lat: ArrayCoordinates1d(lat): Bounds[1.0, 3.0], N[3]
	lon: ArrayCoordinates1d(lon): Bounds[4.0, 7.0], N[4]


Coordinates that have overlapping dimensions cannot be merged, for that we need the `concat` or `union` function.

In [17]:
c1 = podpac.Coordinates([[1, 2, 3]], ['lat'])
c2 = podpac.Coordinates([[4, 5, 6]], ['lat'])

try: 
    podpac.coordinates.merge_dims([c1, c2])
except ValueError as e:
    print (e)

Duplicate dimension 'lat' at position 1


## Concatenating or taking the union of Coordinates
When combinding dimensions:
* `concat` will allow duplicate dimensions
* `union` will only allow unique dimensions, and will also try to sort them

In [18]:
# Create coordinates for example
c1 = podpac.Coordinates([[1, 2, 3, 4]], ['lat'])
c2 = podpac.Coordinates([[4, 5, 6]], ['lat'])

In [19]:
# Concatenate coordinates
concat = podpac.coordinates.concat([c1, c2])

# Order is important
concat_r = podpac.coordinates.concat([c2, c1])

print("Concatenated Coordintes (c1, c2):", concat.coords)
print("Concatenated Coordintes (c2, c1):", concat_r.coords)

Concatenated Coordintes (c1, c2): OrderedDict([('lat', array([1., 2., 3., 4., 4., 5., 6.]))])
Concatenated Coordintes (c2, c1): OrderedDict([('lat', array([4., 5., 6., 1., 2., 3., 4.]))])


In [20]:
# Union of coordinates
union = podpac.coordinates.union([c1, c2])

# Order is not important, due to implicit sorting
union_r = podpac.coordinates.union([c2, c1])

print("Union Coordintes (c1, c2):", union.coords)
print("Union Coordintes (c2, c1):", union_r.coords)

Union Coordintes (c1, c2): OrderedDict([('lat', array([1., 2., 3., 4., 5., 6.]))])
Union Coordintes (c2, c1): OrderedDict([('lat', array([1., 2., 3., 4., 5., 6.]))])
